In [1]:
import os
os.environ["JAX_PLATFORMS"] = "cpu"
import jax.numpy as jnp
from splatmul import matmat, splatsplat, splatmat
from ml_dtypes import bfloat16
import numpy as np

w_size = 1024
hidden_size = 1024
weight_enc, weight_dec = matmat(w_size, hidden_size)
weight_dec = weight_dec.view(bfloat16).copy()
weight_dec = np.ascontiguousarray(weight_dec)
adam_state = splatsplat(w_size, hidden_size,
                        1e-4, 0.9, 0.999, 1e-8, 16)
grad = weight_dec * np.array(1e-3, dtype=bfloat16)

In [6]:
import ctypes


def get_writeable_bit():
    max_arr_size = 128
    test_arr = np.array(1, dtype=np.uint16)
    ptr = id(test_arr)
    str_a = ctypes.string_at(ptr, max_arr_size)
    test_arr.setflags(write=0)
    str_b = ctypes.string_at(ptr, max_arr_size)
    writeable_bit = None
    for i in range(max_arr_size):
        if str_a[i] != str_b[i]:
            byte_a, byte_b = bin(str_a[i])[2:].zfill(8), bin(str_b[i])[2:].zfill(8)
            for j in range(8):
                if byte_a[j] != byte_b[j]:
                    writeable_bit = i * 8 + j
    writeable_byte = writeable_bit // 8, writeable_bit % 8
    return writeable_byte

def transmogrify(x):
    byte, bit = get_writeable_bit()
    ptr = id(x)
    current = ctypes.string_at(ptr + byte, 1)
    target = current[0] | (1 << (7-bit))
    ctypes.memset(ptr + byte, target, 1)
    return x

In [7]:
def jj(x):
    x = jnp.asarray(x).view(jnp.uint16)
    x = np.asarray(x)
    x = transmogrify(x)
    return x
grad_ = jj(grad)
weight_dec_ = jj(weight_dec)

In [8]:
print(np.abs(grad_).max())
splatmat(adam_state, grad_, weight_dec_)
print(np.abs(weight_dec_).max())

47107
asserting layouts
into slice
updating adam
updating m
updating v
updating parameters
all done
48384
